***SINGLE CELL - Data - ANALYSIS - Julia***

In [6]:
using SingleCellProjections, DataFrames, CSV
using SparseArrays, DataFrames, LinearAlgebra
using UMAP, TSne

In [4]:
# function to extract a data from DataMatrix

function f(col, n; h=10, t=10, filler="")
   @assert h+t <= n
   pre = string(col[1:h])
   mid = string("fill(\"", filler, "\", ", n-h-t, ")")
   post = string(col[end-t+1:end])
  string("vcat(", pre, ", ", mid, ", ", post, ')')
end

f (generic function with 1 method)

In [7]:
# function to loadCounts

function load_counts(args...; sample_names, kwargs...) 
    P = 33766
    N = "P1" in sample_names ? 35340 : 42553
    # X = sparse(Int32[], Int32[], Int[], P, N)
    # X = sparse(ones(Int32,N), Int32.(1:N), 1:N, P, N)

    I = repeat(Int32[1,2]; inner=N)
    J = vcat(Int32.(1:N), Int32.(1:N))
    V = vcat(1:N, div.(N:-1:1,3))
    X = sparse(I,J,V,P,N)

    v_id = vcat(["MIR1302-2HG", "FAM138A", "OR4F5", "AL627309.1", "AL627309.3", "AL627309.2", "AL627309.4", "AL732372.1", "OR4F29", "AC114498.1"], string.("dummy_", 1:33746), ["CD169", "CD28", "CD161", "CD163", "CD138-1", "CD164", "CD138-2", "CD144", "CD202b", "CD11c"])
    v_feature_type = vcat(fill("Gene Expression",33538), fill("Antibody Capture", P-33538))

    v_id[33497:33509] .= string.("MT-", 1:13)

    o_id = vcat(["P1_L1_AAACCCAAGACATACA", "P1_L1_AAACCCACATCGGTTA", "P1_L1_AAACCCAGTGGAACAC", "P1_L1_AAACCCATCTGCGGAC", "P1_L1_AAACGAAAGTTACTCG", "P1_L1_AAACGAACAATGAGCG", "P1_L1_AAACGAACACTCCTTG", "P1_L1_AAACGAACAGCATCTA", "P1_L1_AAACGAATCCTCACCA", "P1_L1_AAACGAATCTCACTCG"], string.("dummy_", 1:N-20), ["P2_L5_TTTGGTTGTCCGAAAG", "P2_L5_TTTGGTTTCCTCTAGC", "P2_L5_TTTGGTTTCGTAGGGA", "P2_L5_TTTGGTTTCTTTGATC", "P2_L5_TTTGTTGAGTGTACCT", "P2_L5_TTTGTTGGTACGATCT", "P2_L5_TTTGTTGGTCCTTAAG", "P2_L5_TTTGTTGTCAACACCA", "P2_L5_TTTGTTGTCATGCATG", "P2_L5_TTTGTTGTCCGTGCGA"])
    o_sampleName = vcat(fill("P1",18135), fill("P2", N-18135))
    o_barcode = vcat(["L1_AAACCCAAGACATACA", "L1_AAACCCACATCGGTTA", "L1_AAACCCAGTGGAACAC", "L1_AAACCCATCTGCGGAC", "L1_AAACGAAAGTTACTCG", "L1_AAACGAACAATGAGCG", "L1_AAACGAACACTCCTTG", "L1_AAACGAACAGCATCTA", "L1_AAACGAATCCTCACCA", "L1_AAACGAATCTCACTCG"], string.("dummy_", 1:N-20), ["L5_TTTGGTTGTCCGAAAG", "L5_TTTGGTTTCCTCTAGC", "L5_TTTGGTTTCGTAGGGA", "L5_TTTGGTTTCTTTGATC", "L5_TTTGTTGAGTGTACCT", "L5_TTTGTTGGTACGATCT", "L5_TTTGTTGGTCCTTAAG", "L5_TTTGTTGTCAACACCA", "L5_TTTGTTGTCATGCATG", "L5_TTTGTTGTCCGTGCGA"])

    v = DataFrame(id=v_id, feature_type=v_feature_type, name=v_id, genome="hg19", read="", pattern="", sequence="")
    o = DataFrame(id=o_id, sampleName=o_sampleName, barcode=o_barcode)
    counts = DataMatrix(X, v, o)
end


load_counts (generic function with 1 method)

In [8]:


# SingleCellProjections.sctransform(counts; use_cache=false, verbose=false)
function sctransform(counts)
    m = SCTransformModel(counts; use_cache=false, verbose=false)
    nvar = 20239
    append!(m.params, m.params[mod1.(1:nvar-size(m.params,1),2),:])
    m.params.id = counts.var.id[1:nvar]
    project(counts, m; verbose=false)
end

svd(args...; nsv, kwargs...) = LinearAlgebra.svd(args...; nsv, subspacedims=nsv, niter=1, kwargs...)
force_layout(args...; kwargs...) = SingleCellProjections.force_layout(reduced; niter=1, kwargs...)

umap(args...; kwargs...) = UMAP.umap(args...; n_epochs=1, init=:random, n_neighbors=2, kwargs...)
tsne(data, d; kwargs...) = TSne.tsne(data, d, 0, 1, 5; verbose=false, progress=false, kwargs...) 

tsne (generic function with 1 method)

***Start Analysis...***

In [10]:
## Paths to data
main_path = "JULIA_BIO/scRNA/";
sample_paths = joinpath.(main_path, ["GSE164378_RNA_ADT_3P_P1.h5", "GSE164378_RNA_ADT_3P_P2.h5"]);

counts = load_counts(sample_paths; sample_names=["P1","P2"])

DataMatrix (33766 variables and 35340 observations)
  SparseMatrixCSC{Int64, Int32}
  Variables: id, feature_type, name, genome, read, pattern, sequence
  Observations: id, sampleName, barcode

In [11]:
## Cell Annotation

var_counts_fraction!(counts, "name"=>contains(r"^MT-"), "feature_type"=>isequal("Gene Expression"), "fraction_mt")

DataMatrix (33766 variables and 35340 observations)
  SparseMatrixCSC{Int64, Int32}
  Variables: id, feature_type, name, genome, read, pattern, sequence
  Observations: id, sampleName, barcode, fraction_mt
  Models: VarCountsFractionModel(subset_size=13, total_size=33538, col="fraction_mt")

In [25]:

counts.obs.fraction_mt[1:10] = [194/5864,606/9333,176/3251,299/4198,343/5486,473/7379,196/4444,174/5693,160/4525,156/3519]

csv_str = """barcode,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase,Batch
          L1_AAACCCAAGAAACTCA,7535,217,10823,2915,SeuratProject,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1,Batch1
          L1_AAACCCAAGACATACA,6013,209,5864,1617,SeuratProject,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1,Batch1
          L1_AAACCCACAACTGGTT,6620,213,5067,1381,SeuratProject,L1,P4,2,CD8 T,CD8 Naive,CD8 Naive,S,Batch1
          L1_AAACCCACACGTACTA,3567,202,4786,1890,SeuratProject,L1,P3,7,NK,NK,NK_2,G1,Batch1
          L1_AAACCCACAGCATACT,6402,215,6505,1621,SeuratProject,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1,Batch1
          L1_AAACCCACATCAGTCA,5297,212,4332,1633,SeuratProject,L1,P3,2,CD8 T,CD8 TEM,CD8 TEM_1,G1,Batch1
          L1_AAACCCACATCGGTTA,7634,219,9333,2672,SeuratProject,L1,P1,7,Mono,CD16 Mono,CD16 Mono,G1,Batch1
          L1_AAACCCACATGGATCT,8210,222,3589,1122,SeuratProject,L1,P4,2,B,B intermediate,B intermediate lambda,G1,Batch1
          L1_AAACCCAGTGGAACAC,2847,201,3251,1375,SeuratProject,L1,P1,2,NK,NK,NK_2,G2M,Batch1
          L1_AAACCCATCCACACCT,4557,209,3401,1200,SeuratProject,L1,P3,2,CD8 T,CD8 Naive,CD8 Naive,S,Batch1
          L1_AAACCCATCTGCGGAC,5129,212,4198,1318,SeuratProject,L1,P1,0,CD4 T,CD4 TCM,CD4 TCM_1,S,Batch1
          L1_AAACGAAAGTTACTCG,7630,208,5486,1390,SeuratProject,L1,P1,0,CD4 T,CD4 TCM,CD4 TCM_3,G1,Batch1
          """
cell_annotations = CSV.read(codeunits(csv_str), DataFrame)

Row,barcode,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase,Batch
,String31,Int64,Int64,Int64,Int64,String15,String3,String3,Int64,String7,String15,String31,String3,String7
1,L1_AAACCCAAGAAACTCA,7535,217,10823,2915,SeuratProject,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1,Batch1
2,L1_AAACCCAAGACATACA,6013,209,5864,1617,SeuratProject,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1,Batch1
3,L1_AAACCCACAACTGGTT,6620,213,5067,1381,SeuratProject,L1,P4,2,CD8 T,CD8 Naive,CD8 Naive,S,Batch1
4,L1_AAACCCACACGTACTA,3567,202,4786,1890,SeuratProject,L1,P3,7,NK,NK,NK_2,G1,Batch1
5,L1_AAACCCACAGCATACT,6402,215,6505,1621,SeuratProject,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1,Batch1
6,L1_AAACCCACATCAGTCA,5297,212,4332,1633,SeuratProject,L1,P3,2,CD8 T,CD8 TEM,CD8 TEM_1,G1,Batch1
7,L1_AAACCCACATCGGTTA,7634,219,9333,2672,SeuratProject,L1,P1,7,Mono,CD16 Mono,CD16 Mono,G1,Batch1
8,L1_AAACCCACATGGATCT,8210,222,3589,1122,SeuratProject,L1,P4,2,B,B intermediate,B intermediate lambda,G1,Batch1
9,L1_AAACCCAGTGGAACAC,2847,201,3251,1375,SeuratProject,L1,P1,2,NK,NK,NK_2,G2M,Batch1


In [26]:
leftjoin!(counts.obs, cell_annotations; on=:barcode);

In [27]:
counts.obs[34639+1:end,"celltype.l1"] .= "other"

701-element view(::Vector{Union{Missing, String7}}, 34640:35340) with eltype Union{Missing, String7}:
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 ⋮
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"
 "other"

In [28]:
## Observe some annotations from first few cells:

counts.obs[1:5,["id","sampleName","barcode","fraction_mt","celltype.l1"]]

Row,id,sampleName,barcode,fraction_mt,celltype.l1
,String,String,String,Float64,String7?
1,P1_L1_AAACCCAAGACATACA,P1,L1_AAACCCAAGACATACA,0.0330832,CD4 T
2,P1_L1_AAACCCACATCGGTTA,P1,L1_AAACCCACATCGGTTA,0.0649309,Mono
3,P1_L1_AAACCCAGTGGAACAC,P1,L1_AAACCCAGTGGAACAC,0.0541372,NK
4,P1_L1_AAACCCATCTGCGGAC,P1,L1_AAACCCATCTGCGGAC,0.0712244,CD4 T
5,P1_L1_AAACGAAAGTTACTCG,P1,L1_AAACGAAAGTTACTCG,0.0625228,CD4 T


***TRANSFORMATION & NORMALIZATION***

In [29]:
transformed = sctransform(counts)

DataMatrix (20239 variables and 35340 observations)
  A+B₁B₂B₃
  Variables: id, feature_type, name, genome, read, pattern, sequence, logGeneMean, outlier, beta0, ...
  Observations: id, sampleName, barcode, fraction_mt, nCount_ADT, nFeature_ADT, nCount_RNA, nFeature_RNA, orig.ident, lane, ...
  Models: SCTransformModel(nvar=20239, clip=34.32), VarCountsFraction

In [30]:
normalized = normalize_matrix(transformed, "fraction_mt")

DataMatrix (20239 variables and 35340 observations)
  A+B₁B₂B₃+(-β)X'
  Variables: id, feature_type, name, genome, read, pattern, sequence, logGeneMean, outlier, beta0, ...
  Observations: id, sampleName, barcode, fraction_mt, nCount_ADT, nFeature_ADT, nCount_RNA, nFeature_RNA, orig.ident, lane, ...
  Models: NormalizationModel(rank=2, ~1+num(fraction_mt)), SCTransform, VarCountsFraction

**FILTER (others)***

In [31]:
filtered = filter_obs("celltype.l1"=>!isequal("other"), normalized)

DataMatrix (20239 variables and 34639 observations)
  Aᵣ+B₁B₂B₃ᵣ+(-β)Xₗ'
  Variables: id, feature_type, name, genome, read, pattern, sequence, logGeneMean, outlier, beta0, ...
  Observations: id, sampleName, barcode, fraction_mt, nCount_ADT, nFeature_ADT, nCount_RNA, nFeature_RNA, orig.ident, lane, ...
  Models: FilterModel(:, "celltype.l1"=>!Fix2{typeof(isequal), String}(isequal, "other")), Normalization, SCTransform, VarCountsFraction

***Principal Component Analysis (PCA)***

In [33]:
reduced = svd(filtered; nsv=20)

DataMatrix (20239 variables and 34639 observations)
  SVD (20 dimensions)
  Variables: id, feature_type, name, genome, read, pattern, sequence, logGeneMean, outlier, beta0, ...
  Observations: id, sampleName, barcode, fraction_mt, nCount_ADT, nFeature_ADT, nCount_RNA, nFeature_RNA, orig.ident, lane, ...
  Models: SVDModel(nsv=20), Filter, Normalization, SCTransform, VarCountsFraction

In [34]:
umapped = umap(reduced, 3)

DataMatrix (3 variables and 34639 observations)
  Matrix{Float64}
  Variables: id
  Observations: id, sampleName, barcode, fraction_mt, nCount_ADT, nFeature_ADT, nCount_RNA, nFeature_RNA, orig.ident, lane, ...
  Models: UMAPModel(n_components=3), SVD, Filter, Normalization, SCTransform, ...

In [40]:
using Plots

function plot_categorical_3d(data, annotation; marker_size=3)
    points = obs_coordinates(data)
    
    # Group the data by the annotation column
    groups = groupby(data.obs, annotation)
    
    # Create a plot
    p = plot(; xlabel="X", ylabel="Y", zlabel="Z", legend=:topleft, size=(800, 600))
    
    # Loop through the groups and add each one as a scatter3d trace
    for (value, group) in pairs(groups)
        x = points[1, group.== true]
        y = points[2, group.== true]
        z = points[3, group.== true]
        scatter3d!(p, x, y, z; markersize=marker_size, label=string(value))
    end
    
    return display(p)
end


plot_categorical_3d(reduced, "celltype.l1")

LoadError: ArgumentError: invalid index: 3×17 DataFrame
 Row │ id     sampleName  barcode  fraction_mt  nCount_ADT  nFeature_ADT  nCou ⋯
     │ Bool   Bool        Bool     Bool         Bool        Bool          Bool ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ false       false    false        false       false         false       ⋯
   2 │ false       false    false        false       false         false
   3 │ false       false    false        false       false         false
                                                              11 columns omitted of type DataFrame

In [41]:
display(plot_categorical_3d(reduced, "celltype.l1"))

LoadError: ArgumentError: invalid index: 3×17 DataFrame
 Row │ id     sampleName  barcode  fraction_mt  nCount_ADT  nFeature_ADT  nCou ⋯
     │ Bool   Bool        Bool     Bool         Bool        Bool          Bool ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ false       false    false        false       false         false       ⋯
   2 │ false       false    false        false       false         false
   3 │ false       false    false        false       false         false
                                                              11 columns omitted of type DataFrame